In [31]:
import MDAnalysis as mda
import numpy as np

In [32]:

u = mda.Universe('cecile.gro')
c = np.unique(u.atoms.positions[:,1]) # finding the unique y-coordinate so that we can loop through each row of atoms
u.atoms.masses = 36    # Since it's a TC5 bead, we use a mass of 36 for each

In [33]:
for j in range(len(c)): # Looping through each row, defined by the y-coordinate
    if (j !=0) and (j !=len(c)-1):
        if j % 2 !=0:
            group = u.atoms[u.atoms.positions[:,1] == c[j]]
            gr = np.arange(1, len(group), 3)
         
            for k in gr:
                u.atoms[group[k].index].mass = 0
            
        else:
            group = u.atoms[u.atoms.positions[:,1] == c[j]]
            gr = np.arange(2, len(group), 3)
            for k in gr:
                u.atoms[group[k].index].mass = 0
                

In [34]:
v = u.atoms[u.atoms.masses !=0]

In [35]:
def hexagon(universe):
    
    ''' Identifying the vertics of hexagon around a virtual site '''
    
    b = u.atoms[u.atoms.masses == 0].indices
    hexagon_indices = []
    for i in b:
        empty = []
        for j in u.atoms.indices:
            if (2.55 <= calc_bonds(u.atoms[j].position, u.atoms[i].position) <= 2.57):
                empty.append(j)
        hexagon_indices.append(empty)
    return hexagon_indices

In [36]:
from MDAnalysis.lib.distances import calc_angles, calc_bonds, calc_dihedrals

In [37]:
hexagons = hexagon(u)

In [38]:
print(hexagons)

[[0, 1, 4, 6, 10, 11], [2, 3, 7, 9, 13, 14], [6, 7, 11, 13, 17, 18], [10, 11, 15, 17, 21, 22], [13, 14, 18, 20, 23, 24]]


In [39]:
def get_angles(hexagons):
    angles = []
    for hex in hexagons:
        angle1 = [hex[2], hex[4], hex[0], hex[5]]
        angle2 = [hex[4], hex[0], hex[5], hex[1]]
        angle3 = [hex[0], hex[5], hex[1], hex[3]]
        angles.append(angle1)
        angles.append(angle2)
        angles.append(angle3)
    return angles

In [40]:
angles = get_angles(hexagons)

In [41]:
angles

[[4, 10, 0, 11],
 [10, 0, 11, 1],
 [0, 11, 1, 6],
 [7, 13, 2, 14],
 [13, 2, 14, 3],
 [2, 14, 3, 9],
 [11, 17, 6, 18],
 [17, 6, 18, 7],
 [6, 18, 7, 13],
 [15, 21, 10, 22],
 [21, 10, 22, 11],
 [10, 22, 11, 17],
 [18, 23, 13, 24],
 [23, 13, 24, 14],
 [13, 24, 14, 20]]

In [42]:
def get_between_hexagons_angles(hexagons, beads_per_row):
    def get_common_side(hexa1, hexa2):
        common = list(set(hexa1).intersection(set(hexa2)))
        common.sort()
        return common
    
    angles = []
    gap_for_above = int(beads_per_row / 2)
    
    for i in range(len(hexagons)):
        hexa = hexagons[i]
        try:
            hexa_after = hexagons[i+2]
        except IndexError:
            hexa_after = None
        try: 
            hexa_above = hexagons[i+gap_for_above]
        except IndexError:
            hexa_above = None
        
        try: 
            hexa_before = hexagons[i - 1]
        except IndexError:
            hexa_before = None
            
        
        if hexa_above == hexa_before: 
            hexa_before = None
        
        if hexa_after:
            side_after = get_common_side(hexa, hexa_after)
            if side_after:
                idx1 = hexa.index(side_after[0])
                idx2 = hexa_after.index(side_after[1])
                angle = [hexa[idx1 - 2], side_after[0], side_after[1], hexa_after[idx2 + 2]]
                angles.append(angle)
            
        
        if hexa_above: 
            side_above = get_common_side(hexa, hexa_above)
            if side_above: 
                idx1 = hexa.index(side_above[0])
                idx2 = hexa_above.index(side_above[1])
                angle = [hexa[idx1 - 4], side_above[0], side_above[1], hexa_above[idx2 + 4]]
                angles.append(angle)  
                
        if hexa_before:
            side_before = get_common_side(hexa, hexa_before)
            if side_before:
                idx1 = hexa_before.index(side_before[0])
                idx2 = hexa.index(side_before[1])
                angle = [hexa_before[idx1 - 2], side_before[0], side_before[1], hexa[idx2 + 2]]
                angles.append(angle)
    return angles

In [44]:
other_angles = get_between_hexagons_angles(hexagons, 6)

In [45]:
other_angles

[[1, 6, 11, 17],
 [0, 10, 11, 22],
 [2, 13, 14, 24],
 [7, 13, 18, 23],
 [2, 7, 13, 18],
 [6, 11, 17, 22]]

In [46]:
angles

[[4, 10, 0, 11],
 [10, 0, 11, 1],
 [0, 11, 1, 6],
 [7, 13, 2, 14],
 [13, 2, 14, 3],
 [2, 14, 3, 9],
 [11, 17, 6, 18],
 [17, 6, 18, 7],
 [6, 18, 7, 13],
 [15, 21, 10, 22],
 [21, 10, 22, 11],
 [10, 22, 11, 17],
 [18, 23, 13, 24],
 [23, 13, 24, 14],
 [13, 24, 14, 20]]

In [47]:
import numpy as np

In [48]:
collect = np.vstack((angles, other_angles))

In [49]:
collect

array([[ 4, 10,  0, 11],
       [10,  0, 11,  1],
       [ 0, 11,  1,  6],
       [ 7, 13,  2, 14],
       [13,  2, 14,  3],
       [ 2, 14,  3,  9],
       [11, 17,  6, 18],
       [17,  6, 18,  7],
       [ 6, 18,  7, 13],
       [15, 21, 10, 22],
       [21, 10, 22, 11],
       [10, 22, 11, 17],
       [18, 23, 13, 24],
       [23, 13, 24, 14],
       [13, 24, 14, 20],
       [ 1,  6, 11, 17],
       [ 0, 10, 11, 22],
       [ 2, 13, 14, 24],
       [ 7, 13, 18, 23],
       [ 2,  7, 13, 18],
       [ 6, 11, 17, 22]])

In [50]:
collect = collect + 1

In [51]:
collect

array([[ 5, 11,  1, 12],
       [11,  1, 12,  2],
       [ 1, 12,  2,  7],
       [ 8, 14,  3, 15],
       [14,  3, 15,  4],
       [ 3, 15,  4, 10],
       [12, 18,  7, 19],
       [18,  7, 19,  8],
       [ 7, 19,  8, 14],
       [16, 22, 11, 23],
       [22, 11, 23, 12],
       [11, 23, 12, 18],
       [19, 24, 14, 25],
       [24, 14, 25, 15],
       [14, 25, 15, 21],
       [ 2,  7, 12, 18],
       [ 1, 11, 12, 23],
       [ 3, 14, 15, 25],
       [ 8, 14, 19, 24],
       [ 3,  8, 14, 19],
       [ 7, 12, 18, 23]])